In [1]:
# LOAD PACKAGES
# -------------
using JuMP        # Need to say it whenever we use JuMP
using GLPK        # Loading the GLPK module for using its solver
using DataFrames  # Julia library for working with tabular data 
using CSV         # Julia library for handling delimited text files

# MODEL CONSTRUCTION
# ------------------
capstone = Model(GLPK.Optimizer) 
set_time_limit_sec(capstone::Model, 60) # We also set the time limit, in seconds

In [2]:
# INPUT DATA
# ----------

# Data on student preferences
Rank = DataFrame(CSV.File("Rank.csv",header=false))
print(size(Rank))

# Data on Pillars
Pillar = DataFrame(CSV.File("Pillar.csv",header=false))
print(size(Pillar))

# Data on Lower and Upper Bound (min and max number of students needed for each project from each discipline)
LowerBound = DataFrame(CSV.File("LowerBound.csv",header=false))
UpperBound = DataFrame(CSV.File("UpperBound.csv",header=false))
print(size(LowerBound))
print(size(UpperBound))

# Other data
Students = 1:170 # students
Projects = 1:61  # projects
PillarId = 1:4   # pillars

(170, 61)(170, 4)(4, 61)(4, 61)

1:4

In [3]:
# VARIABLES
# ---------

# Binary variable taking value 1 if the s student is allocated to project p
@variable(capstone, x[Students,Projects], Bin)  

# Binary variable taking value 1 if project p is launched
@variable(capstone, y[Projects], Bin)          

1-dimensional DenseAxisArray{VariableRef,1,...} with index sets:
    Dimension 1, 1:61
And data, a 61-element Vector{VariableRef}:
 y[1]
 y[2]
 y[3]
 y[4]
 y[5]
 y[6]
 y[7]
 y[8]
 y[9]
 y[10]
 y[11]
 y[12]
 y[13]
 ⋮
 y[50]
 y[51]
 y[52]
 y[53]
 y[54]
 y[55]
 y[56]
 y[57]
 y[58]
 y[59]
 y[60]
 y[61]

In [4]:
# CONSTRAINTS
# -----------

# Every student is allocated to a single project
@constraint(capstone, allocatestudent[i=Students], sum(x[i,j] for j = Projects)==1)

# A student is allocated to a project only if the project is launched
@constraint(capstone, allocateifoffered[i=Students,j=Projects],x[i,j]<=y[j])

# Number of student of the different discplines allocated to a project is within the min. and upp. bound
@constraint(capstone, lower[j=Projects,k=PillarId],sum(x[i,j]*Pillar[i,k] for i = Students) >= LowerBound[k,j]*y[j])
@constraint(capstone, upper[j=Projects,k=PillarId],sum(x[i,j]*Pillar[i,k] for i = Students) <= UpperBound[k,j]*y[j])

2-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},2,...} with index sets:
    Dimension 1, 1:61
    Dimension 2, 1:4
And data, a 61×4 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 upper[1,1] : x[1,1] + x[2,1] + x[3,1] + x[4,1] + x[5,1] + x[6,1] + x[7,1] + x[8,1] + x[9,1] + x[10,1] + x[11,1] + x[12,1] + x[13,1] + x[14,1] + x[15,1] + x[16,1] + x[17,1] + x[18,1] + x[19,1] + x[20,1] + x[21,1] + x[22,1] + x[23,1] + x[24,1] + x[25,1] + x[26,1] + x[27,1] + x[28,1] + x[29,1] + x[30,1] + x[31,1] + x[32,1] + x[33,1] + x[34,1] + x[35,1] + x[36,1] + x[37,1] + x[38,1] + x[39,1] + x[40,1] + x[41,1] + x[42,1] + x[43,1] + x[44,1] + x[45,1] + x[46,1] + x[47,1] + x[48,1] + x[49,1] + x[50,1] + x[51,1] + x[52,1] + x[53,1] + x[54,1] + x[55,1] + x[56,1] + x[57,1] +

In [5]:
# OBJECTIVE
# ---------

# Maximize the total utility
@objective(capstone, Max, sum(Rank[i,j]*x[i,j] for i = Students, j = Projects))

# PRINT THE MODEL IN A HUMAN-READABLE FORMAT
# ------------------------------------------
# println("The optimization problem to be solved is:")
# print(capstone) # Shows the model constructed in a human-readable form

3 x[1,1] + 4 x[1,8] + x[1,9] + 2 x[1,14] + 8 x[1,20] + 5 x[1,23] + 6 x[1,31] + 9 x[1,37] + 7 x[1,42] + 10 x[1,54] + 7 x[2,1] + 5 x[2,7] + 6 x[2,12] + 3 x[2,14] + 8 x[2,20] + 9 x[2,22] + 4 x[2,25] + 10 x[2,37] + x[2,54] + 2 x[2,58] + x[3,2] + 5 x[3,7] + 7 x[3,8] + 8 x[3,9] + 4 x[3,14] + 3 x[3,30] + 6 x[3,31] + 10 x[3,37] + 9 x[3,54] + 2 x[3,58] + 4 x[4,4] + 10 x[4,7] + 5 x[4,8] + 6 x[4,9] + 9 x[4,10] + 2 x[4,23] + 3 x[4,37] + 8 x[4,40] + 7 x[4,46] + x[4,54] + 8 x[5,7] + 4 x[5,8] + 9 x[5,9] + 7 x[5,14] + 6 x[5,25] + 3 x[5,30] + 2 x[5,31] + 10 x[5,37] + x[5,54] + 5 x[5,58] + 8 x[6,7] + 9 x[6,8] + 10 x[6,9] + 5 x[6,21] + 3 x[6,30] + 4 x[6,33] + 7 x[6,37] + 6 x[6,40] + 2 x[6,46] + x[6,57] + 4 x[7,1] + 10 x[7,7] + 2 x[7,8] + 7 x[7,9] + 3 x[7,10] + 5 x[7,14] + 9 x[7,25] + x[7,31] + 6 x[7,46] + 8 x[7,58] + 10 x[8,1] + 8 x[8,7] + 7 x[8,8] + 9 x[8,12] + 3 x[8,21] + 2 x[8,22] + 4 x[8,23] + x[8,25] + 6 x[8,30] + 5 x[8,31] + 5 x[9,1] + 6 x[9,8] + 10 x[9,9] + 2 x[9,10] + 4 x[9,14] + 9 x[9,30] + 8 x[

In [6]:
# SOLVE IT
# --------

status = optimize!(capstone)
termination_status(capstone)

TIME_LIMIT::TerminationStatusCode = 12

In [7]:
# DISPLAY THE RESULTS
# -------------------

println("Objective value: ", objective_value(capstone)) 
println("The average utility is: ", objective_value(capstone)/maximum(Students)) 
println("The total number of projects is: ", sum(value.(y))) 

Objective value: 1448.9999999999998
The average utility is: 8.523529411764704
The total number of projects is: 29.0
